In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---

# Mouse

## Macosko

In [ ]:
macosko = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Macosko/data.h5")
utils.peek(macosko, "build/retina/Macosko")
macosko.obs.head()

In [ ]:
macosko.obs["cluster"] = "cluster_" + macosko.obs["cluster"].astype(int).astype(str)
macosko.obs.dtypes

In [ ]:
macosko_model = cb.directi.fit_DIRECTi(
    macosko, macosko.uns["seurat_genes"],
    **fixed_model_kwargs
)
macosko.latent = macosko_model.inference(macosko)

In [ ]:
ax = macosko.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Macosko/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macosko.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Macosko/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = macosko.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Macosko/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
macosko.write_dataset("build/retina/Macosko/Macosko.h5")

In [ ]:
%%capture capio
macosko_models = [macosko_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    macosko_models.append(cb.directi.fit_DIRECTi(
        macosko, macosko.uns["seurat_genes"],
        **fixed_model_kwargs,
        random_seed=i
    ))
macosko_blast = cb.blast.BLAST(macosko_models, macosko)
macosko_blast.save("build/retina/Macosko")

In [ ]:
with open("build/retina/Macosko/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/Macosko/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(macosko_blast, "build/retina/Macosko")

In [ ]:
%%writefile build/retina/Macosko/predictable.txt
cell_ontology_class
cell_type1
cluster

## Shekhar

In [ ]:
shekhar = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Shekhar/data.h5")
utils.peek(shekhar, "build/retina/Shekhar")
shekhar.obs.head()

In [ ]:
shekhar.obs["cluster"] = "cluster_" + shekhar.obs["cluster"].astype(int).astype(str)
shekhar.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="batch")
shekhar_model = cb.directi.fit_DIRECTi(
    shekhar, shekhar.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
shekhar.latent = shekhar_model.inference(shekhar)

In [ ]:
ax = shekhar.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Shekhar/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = shekhar.visualize_latent("cell_type2", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Shekhar/cell_type2.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = shekhar.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Shekhar/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = shekhar.visualize_latent("batch", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Shekhar/batch.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = shekhar.visualize_latent("replicate", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Shekhar/replicate.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
shekhar.write_dataset("build/retina/Shekhar/Shekhar.h5")

In [ ]:
%%capture capio
shekhar_models = [shekhar_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    shekhar_models.append(cb.directi.fit_DIRECTi(
        shekhar, shekhar.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
shekhar_blast = cb.blast.BLAST(shekhar_models, shekhar)
shekhar_blast.save("build/retina/Shekhar")

In [ ]:
with open("build/retina/Shekhar/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/Shekhar/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(shekhar_blast, "build/retina/Shekhar")

In [ ]:
%%writefile build/retina/Shekhar/predictable.txt
cell_ontology_class
cell_type1
cell_type2
cluster

## Mouse aligned

In [ ]:
if not os.path.exists("build/retina/ALIGNED_Mus_musculus_Retina"):
    os.makedirs("build/retina/ALIGNED_Mus_musculus_Retina")
shekhar.obs["shekhar_batch"] = shekhar.obs["batch"]
mouse = cb.data.ExprDataSet.merge_datasets(dict(
    macosko=macosko, shekhar=shekhar
), merge_uns_slots=["seurat_genes"])

In [ ]:
mouse.obs.dtypes

In [ ]:
opt_model_kwargs=dict(
    batch_effect=["dataset_name", "shekhar_batch"],
    rmbatch_module_kwargs=[dict(lambda_reg=0.001), dict(lambda_reg=0.005)]
)
mouse_model = cb.directi.fit_DIRECTi(
    mouse, mouse.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
mouse.latent = mouse_model.inference(mouse)

In [ ]:
ax = mouse.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Mus_musculus_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Mus_musculus_Retina/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = mouse.visualize_latent("shekhar_batch", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Mus_musculus_Retina/shekhar_batch.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
mouse.write_dataset("build/retina/ALIGNED_Mus_musculus_Retina/ALIGNED_Mus_musculus_Retina.h5")

In [ ]:
%%capture capio
mouse_models = [mouse_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    mouse_models.append(cb.directi.fit_DIRECTi(
        mouse, mouse.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
mouse_blast = cb.blast.BLAST(mouse_models, mouse)
mouse_blast.save("build/retina/ALIGNED_Mus_musculus_Retina")

In [ ]:
with open("build/retina/ALIGNED_Mus_musculus_Retina/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/ALIGNED_Mus_musculus_Retina/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(mouse_blast, "build/retina/ALIGNED_Mus_musculus_Retina")

In [ ]:
%%writefile build/retina/ALIGNED_Mus_musculus_Retina/predictable.txt
cell_ontology_class

---

# Human

## Lukowski

In [ ]:
lukowski = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Lukowski/data.h5")
utils.peek(lukowski, "build/retina/Lukowski")
lukowski.obs.head()

In [ ]:
lukowski.obs.dtypes

In [ ]:
lukowski.uns["scanpy_genes"].shape

In [ ]:
lukowski_model = cb.directi.fit_DIRECTi(
    lukowski, lukowski.uns["scanpy_genes"],
    **fixed_model_kwargs
)
lukowski.latent = lukowski_model.inference(lukowski)

In [ ]:
ax = lukowski.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Lukowski/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = lukowski.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/Lukowski/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
lukowski.write_dataset("build/retina/Lukowski/Lukowski.h5")

In [ ]:
%%capture capio
lukowski_models = [lukowski_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    lukowski_models.append(cb.directi.fit_DIRECTi(
        lukowski, lukowski.uns["seurat_genes"],
        **fixed_model_kwargs,
        random_seed=i
    ))
lukowski_blast = cb.blast.BLAST(lukowski_models, lukowski)
lukowski_blast.save("build/retina/Lukowski")

In [ ]:
with open("build/retina/Lukowski/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/Lukowski/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(lukowski_blast, "build/retina/Lukowski")

In [ ]:
%%writefile build/retina/Lukowski/predictable.txt
cell_ontology_class
cell_type1

## Liang

In [ ]:
liang = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Liang/data.h5")
utils.peek(liang, "build/retina/Liang")
liang.obs.head()

In [ ]:
liang.obs.dtypes

In [ ]:
liang.uns["scanpy_genes"].shape

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
liang_model = cb.directi.fit_DIRECTi(
    liang, liang.uns["scanpy_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
liang.latent = liang_model.inference(liang)

In [ ]:
# 3196 genes
ax = liang.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True), method="UMAP")
ax.get_figure().savefig("build/retina/Liang/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = liang.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True), method="UMAP")
ax.get_figure().savefig("build/retina/Liang/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = liang.visualize_latent("donor", scatter_kws=dict(rasterized=True), method="UMAP")
ax.get_figure().savefig("build/retina/Liang/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = liang.visualize_latent("region", scatter_kws=dict(rasterized=True), method="UMAP")
ax.get_figure().savefig("build/retina/Liang/region.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
liang.write_dataset("build/retina/Liang/Liang.h5")

In [ ]:
%%capture capio
liang_models = [liang_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    liang_models.append(cb.directi.fit_DIRECTi(
        liang, liang.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
liang_blast = cb.blast.BLAST(liang_models, liang)
liang_blast.save("build/retina/Liang")

In [ ]:
with open("build/retina/Liang/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/Liang/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(liang_blast, "build/retina/Liang")

In [ ]:
%%writefile build/retina/Liang/predictable.txt
cell_ontology_class
cell_type1

## Human aligned

In [ ]:
if not os.path.exists("build/retina/ALIGNED_Homo_sapiens_Retina"):
    os.makedirs("build/retina/ALIGNED_Homo_sapiens_Retina")
liang.obs["liang_donor"] = liang.obs["donor"]
human = cb.data.ExprDataSet.merge_datasets(dict(
    lukowski=lukowski, liang=liang
), merge_uns_slots=["scanpy_genes"])

In [ ]:
human.obs.dtypes

In [ ]:
opt_model_kwargs=dict(
    batch_effect=["dataset_name", "liang_donor"]#,
#     rmbatch_module_kwargs=[dict(lambda_reg=0.001), dict(lambda_reg=0.005)]
)
human_model = cb.directi.fit_DIRECTi(
    human, human.uns["scanpy_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
human.latent = human_model.inference(human)

In [ ]:
# new
ax = human.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
# ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
# new
ax = human.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
# ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
# new
ax = human.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
# ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
# new
ax = human.visualize_latent("liang_donor", scatter_kws=dict(rasterized=True))
# ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = human.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = human.visualize_latent("dataset_name", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/dataset_name.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = human.visualize_latent("liang_donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/retina/ALIGNED_Homo_sapiens_Retina/shekhar_batch.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
human.write_dataset("build/retina/ALIGNED_Homo_sapiens_Retina/ALIGNED_Homo_sapiens_Retina.h5")

In [ ]:
%%capture capio
human_models = [human_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    human_models.append(cb.directi.fit_DIRECTi(
        human, human.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
human_blast = cb.blast.BLAST(human_models, human)
human_blast.save("build/retina/ALIGNED_Homo_sapiens_Retina")

In [ ]:
with open("build/retina/ALIGNED_Homo_sapiens_Retina/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/retina/ALIGNED_Homo_sapiens_Retina/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(human_blast, "build/retina/ALIGNED_Homo_sapiens_Retina")

In [ ]:
%%writefile build/retina/ALIGNED_Homo_sapiens_Retina/predictable.txt
cell_ontology_class